# CS 224D Assignment #2
# Part [2]: Recurrent Neural Networks

This notebook will provide starter code, testing snippets, and additional guidance for implementing the Recurrent Neural Network Language Model (RNNLM) described in Part 2 of the handout.

Please complete parts (a), (b), and (c) of Part 2 before beginning this notebook.

##Malaa: (a) Perplexity

* Cross entropy $$J^{t}(\theta) = - \sum_j y_j^{t} \log \hat y_j^t = - \log \hat y_k^t = \log \frac{1}{\hat y_k^t}$$ where $k$ is the true output word
  
  
* Perplexity $$PP^t (\theta) = \frac{1}{\sum_j y_j^t \hat y_j^t} = \frac{1}{\hat y_k^t}$$
  
  
* Relation: $$PP^t(\theta)=\exp\left(J^t(\theta) \right)$$ so summing cross entropy is like multiplying perplexities

##Malaa: (b) Gradients


* $U$: $$\frac{\partial J^t}{\partial U}=(\hat y - y) {h^{t}}^T = \delta_1^t {h^t}^T$$ where $\delta_1^t = \hat y^t - y^t \in R^{|V|\times1}$ and $h^t \in R^{D_h\times1}$  
  
  
* $H$: $$\left. \frac{\partial J^t}{\partial H} \right|_{t}=\left( U^T \delta_1^t  \circ \sigma'(h^t) \right) {h^{t-1}}^T =\delta^t {h^{t-1}}^T$$ where $$\delta^t = U^T \delta_1^t  \circ \sigma'(h^t) \in R^{D_h\times1}$$ and $\sigma'(x)=\sigma(x) \left( 1 - \sigma(x) \right)$ is the derivative of the sigmoid function.
  
  
* $h^{t-1}$: $$\frac{\partial J^t}{\partial h^{t-1}}=H^T \delta^t $$
  
  
* $L_{x^t}$: $$\frac{\partial J^t}{\partial L_{x^t}}=\delta^t$$
  


##Malaa: (c) Gradients Through Time

* $H$: $$\left. \frac{\partial J^t}{\partial H} \right|_{t-1}=\left( H^T \delta^t  \circ \sigma'(h^{t-1}) \right) {h^{t-2}}^T =\delta^{t-1} {h^{t-2}}^T$$ where $$\delta^{t-1} = H^T \delta^t  \circ \sigma'(h^{t-1}) \in R^{D_h\times1}$$ 
  
  
* $L_{x^{t-1}}$: $$\frac{\partial J^t}{\partial L_{x^{t-1}}}=\delta^{t-1}$$  

More generally for step $t-k$:
  
* $\delta^{t-k}$: $$\delta^{t-k} = H^T \delta^{t-k+1}  \circ \sigma'(h^{t-k}) \in R^{D_h\times1}$$ 
  

* $H$: $$\left. \frac{\partial J^t}{\partial H} \right|_{t-k}=\delta^{t-k} {h^{t-k-1}}^T$$
  
  
* $L_{x^{t-k}}$: $$\frac{\partial J^t}{\partial L_{x^{t-k}}}=\delta^{t-k}$$  

  

##Malaa: (d) 

In [2]:
import sys, os
from numpy import *
from matplotlib.pyplot import *
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 100

%load_ext autoreload
%autoreload 2

## (e): Implement a Recurrent Neural Network Language Model

Follow the instructions on the handout to implement your model in `rnnlm.py`, then use the code below to test.

In [13]:
from rnnlm import RNNLM
# Gradient check on toy data, for speed
random.seed(10)
wv_dummy = random.randn(10,50)
model = RNNLM(L0 = wv_dummy, U0 = wv_dummy,
              alpha=0.005, rseed=10, bptt=4)
model.grad_check(array([1,2,3]), array([2,3,4]))

NOTE: temporarily setting self.bptt = len(y) = 3 to compute true gradient.
grad_check: dJ/dH error norm = 3.014e-09 [ok]
    H dims: [50L, 50L] = 2500 elem
grad_check: dJ/dU error norm = 3.935e-10 [ok]
    U dims: [10L, 50L] = 500 elem
grad_check: dJ/dL[1] error norm = 1.097e-09 [ok]
    L[1] dims: [50L] = 50 elem
grad_check: dJ/dL[2] error norm = 8.78e-10 [ok]
    L[2] dims: [50L] = 50 elem
grad_check: dJ/dL[3] error norm = 1.155e-09 [ok]
    L[3] dims: [50L] = 50 elem
Reset self.bptt = 4


## Prepare Vocabulary and Load PTB Data

We've pre-prepared a list of the vocabulary in the Penn Treebank, along with their absolute counts and unigram frequencies. The document loader code below will "canonicalize" words and replace any unknowns with a `"UUUNKKK"` token, then convert the data to lists of indices.

In [14]:
from data_utils import utils as du
import pandas as pd

# Load the vocabulary
vocab = pd.read_table("data/lm/vocab.ptb.txt", header=None, sep="\s+",
                     index_col=0, names=['count', 'freq'], )

# Choose how many top words to keep
vocabsize = 2000
num_to_word = dict(enumerate(vocab.index[:vocabsize]))
word_to_num = du.invert_dict(num_to_word)
##
# Below needed for 'adj_loss': DO NOT CHANGE
fraction_lost = float(sum([vocab['count'][word] for word in vocab.index
                           if (not word in word_to_num) 
                               and (not word == "UUUNKKK")]))
fraction_lost /= sum([vocab['count'][word] for word in vocab.index
                      if (not word == "UUUNKKK")])
print "Retained %d words from %d (%.02f%% of all tokens)" % (vocabsize, len(vocab),
                                                             100*(1-fraction_lost))

Retained 2000 words from 38444 (84.00% of all tokens)


Load the datasets, using the vocabulary in `word_to_num`. Our starter code handles this for you, and also generates lists of lists X and Y, corresponding to input words and target words*. 

*(Of course, the target words are just the input words, shifted by one position, but it can be cleaner and less error-prone to keep them separate.)*

In [15]:
# Load the training set
docs = du.load_dataset('data/lm/ptb-train.txt')
S_train = du.docs_to_indices(docs, word_to_num)
X_train, Y_train = du.seqs_to_lmXY(S_train)

# Load the dev set (for tuning hyperparameters)
docs = du.load_dataset('data/lm/ptb-dev.txt')
S_dev = du.docs_to_indices(docs, word_to_num)
X_dev, Y_dev = du.seqs_to_lmXY(S_dev)

# Load the test set (final evaluation only)
docs = du.load_dataset('data/lm/ptb-test.txt')
S_test = du.docs_to_indices(docs, word_to_num)
X_test, Y_test = du.seqs_to_lmXY(S_test)

# Display some sample data
print " ".join(d[0] for d in docs[7])
print S_test[7]

Big investment banks refused to step up to the plate to support the beleaguered floor traders by buying big blocks of stock , traders say .
[   4  147  169  250 1879    7 1224   64    7    1    3    7  456    1    3
 1024  255   24  378  147    3    6   67    0  255  138    2    5]


## (f): Train and evaluate your model

When you're able to pass the gradient check, let's run our model on some real language!

You should randomly initialize the word vectors as Gaussian noise, i.e. $L_{ij} \sim \mathit{N}(0,0.1)$ and $U_{ij} \sim \mathit{N}(0,0.1)$; the function `random.randn` may be helpful here.

As in Part 1, you should tune hyperparameters to get a good model.

In [19]:
hdim = 100 # dimension of hidden layer = dimension of word vectors
random.seed(10)
L0 = zeros((vocabsize, hdim)) # replace with random init, 
                              # or do in RNNLM.__init__()
L0 = random.randn(vocabsize, hdim) * 0.1    
# test parameters; you probably want to change these
model = RNNLM(L0, U0 = L0, alpha=0.01, rseed=10, bptt=5)

# Gradient check is going to take a *long* time here
# since it's quadratic-time in the number of parameters.
# run at your own risk... (but do check this!)
# model.grad_check(array([1,2,3]), array([2,3,4]))

In [20]:
#### YOUR CODE HERE ####

##
# Pare down to a smaller dataset, for speed
# (optional - recommended to not do this for your final model)
ntrain = len(Y_train)
X = X_train[:ntrain]
Y = Y_train[:ntrain]

# minibatch schedule
k = 5
idx_mb = model.randomiter(ntrain/k, ntrain, batch=k)

# train
costs = model.train_sgd(X, Y, idxiter=idx_mb, alphaiter=None, printevery=1000, costevery=1000, devidx=None)


#### END YOUR CODE ####

Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 7.6239
  Seen 1000 in 514.82 s
  [1000]: mean loss 5.02483
  Seen 2000 in 1029.01 s
  [2000]: mean loss 4.91929
  Seen 3000 in 1522.97 s
  [3000]: mean loss 4.81859
  Seen 4000 in 2026.98 s
  [4000]: mean loss 4.73009
  Seen 5000 in 2534.95 s
  [5000]: mean loss 4.67345
  Seen 6000 in 3062.14 s
  [6000]: mean loss 4.62386
  Seen 7000 in 3579.88 s
  [7000]: mean loss 4.56981
  Seen 8000 in 4108.21 s
  [8000]: mean loss 4.55673
  Seen 9000 in 4636.04 s
  [9000]: mean loss 4.49353
  Seen 10000 in 5168.53 s
  [10000]: mean loss 4.47091
  Seen 11000 in 5704.29 s
  [11000]: mean loss 4.43376
  [11304]: mean loss 4.43525
SGD complete: 11304 examples in 6464.42 seconds.


In [25]:
## Evaluate cross-entropy loss on the dev set,
## then convert to perplexity for your writeup
dev_loss = model.compute_mean_loss(X_dev, Y_dev)
print dev_loss

4.46054721157


The performance of the model is skewed somewhat by the large number of `UUUNKKK` tokens; if these are 1/6 of the dataset, then that's a sizeable fraction that we're just waving our hands at. Naively, our model gets credit for these that's not really deserved; the formula below roughly removes this contribution from the average loss. Don't worry about how it's derived, but do report both scores - it helps us compare across models with different vocabulary sizes.

In [22]:
## DO NOT CHANGE THIS CELL ##
# Report your numbers, after computing dev_loss above.
def adjust_loss(loss, funk, q, mode='basic'):
    if mode == 'basic':
        # remove freebies only: score if had no UUUNKKK
        return (loss + funk*log(funk))/(1 - funk)
    else:
        # remove freebies, replace with best prediction on remaining
        return loss + funk*log(funk) - funk*log(q)
# q = best unigram frequency from omitted vocab
# this is the best expected loss out of that set
q = vocab.freq[vocabsize] / sum(vocab.freq[vocabsize:])
print "Unadjusted: %.03f" % exp(dev_loss)
print "Adjusted for missing vocab: %.03f" % exp(adjust_loss(dev_loss, fraction_lost, q))

Unadjusted: 86.535
Adjusted for missing vocab: 142.730


### Save Model Parameters

In [23]:
##
# Save to .npy files; should only be a few MB total
assert(min(model.sparams.L.shape) <= 100) # don't be too big
assert(max(model.sparams.L.shape) <= 5000) # don't be too big
save("rnnlm.L.npy", model.sparams.L)
save("rnnlm.U.npy", model.params.U)
save("rnnlm.H.npy", model.params.H)

## (g): Generating Data

Once you've trained your model to satisfaction, let's use it to generate some sentences!

Implement the `generate_sequence` function in `rnnlm.py`, and call it below.

In [24]:
def seq_to_words(seq):
    return [num_to_word[s] for s in seq]
    
seq, J = model.generate_sequence(word_to_num["<s>"], 
                                 word_to_num["</s>"], 
                                 maxlen=100)
print J
# print seq
print " ".join(seq_to_words(seq))

34.6967568984
<s> u.s. through the UUUNKKK auto in UUUNKKK . </s>


**BONUS:** Use the unigram distribution given in the `vocab` table to fill in any `UUUNKKK` tokens in your generated sequences with words that we omitted from the vocabulary. You'll want to use `list(vocab.index)` to get a list of words, and `vocab.freq` to get a list of corresponding frequencies.

In [20]:
# Replace UUUNKKK with a random unigram,
# drawn from vocab that we skipped
from nn.math import MultinomialSampler, multinomial_sample
def fill_unknowns(words):
    #### YOUR CODE HERE ####
    ret = words # do nothing; replace this
    

    #### END YOUR CODE ####
    return ret
    
print " ".join(fill_unknowns(seq_to_words(seq)))